# TP : canal de diffusion anonyme et génération de secret
### Antoine Maurais et Florian Monfort – IAI 3

Ce TP va consister à mettre en œuvre la création d’un secret partagé entre deux complices sur un canal de diffusion anonyme à partir de messages publics.

In [92]:
from datetime import datetime, timedelta
from time import sleep
from typing import NamedTuple
import random

## Partie 1

Dans cette première partie, on crée une classe décrivant un canal de diffusion anonyme et on y met les deux méthodes voulues :

In [93]:
class Message(NamedTuple):
    contenu: str
    date: datetime


class Canal:
    file_messages: list[Message] = []

    def poster_message_anonyme(self, message: str):
        message = Message(message, datetime.now())
        self.file_messages.append(message)
        return message

    def recuperer_messages_anonymes(self, debut: datetime, fin: datetime):
        return [message for message in self.file_messages if debut <= message.date <= fin]

Note : l’algorithme choisi pour la récupération des messages anonymes n’est clairement pas optimal à cause de la structure de données choisie, mais faire mieux n’est pas le but de l’exercice.

## Partie 2

In [94]:
DELAI_MIN_MS = 1
DELAI_MAX_MS = 10


def generer_secret(canal: Canal, complice_a: str, complice_b: str, duree_ms: int):
    debut = datetime.now()
    messages_complice_a = []
    messages_complice_b = []

    # Tant que le délai n'est pas écoulé
    while (datetime.now() - debut).total_seconds() * 1_000 < duree_ms:
        # Génère un message contenant aléatoirement le nom du complice A ou du complice B et l'envoie sur le canal
        message = canal.poster_message_anonyme(complice_a if random.random() < 0.5 else complice_b)

        # Puis l'enregistre au nom du complice A ou du complice B avec une chance équiprobable
        if random.random() < 0.5:
            messages_complice_a.append(message)
        else:
            messages_complice_b.append(message)

        # Laisse un délai aléatoire s'écouler avant le prochain message
        sleep(random.randrange(DELAI_MIN_MS, DELAI_MAX_MS) / 1_000)

    return (messages_complice_a, messages_complice_b)


def extraire_secret(canal: Canal, complice: str, messages_complice: list[Message], debut: datetime, duree_ms: int):
    liste_messages = canal.recuperer_messages_anonymes(debut, debut + timedelta(milliseconds=duree_ms))

    secret = ""
    for message in liste_messages:
        # Ajoute 0 au secret si le contenu du message est le nom de ce complice et qu'il l'a écrit, ou contient autre chose et qu'il ne l'a pas écrit
        if (message.contenu == complice) == (message in messages_complice):
            secret += "0"
        # Dans tous les autres cas, ajoute 1
        else:
            secret += "1"

    return secret

On exécute notre programme et on affiche les secrets obtenus par Alice et Bob. On s’assure ainsi que le résultat qu’ils obtiennent est identique :

In [95]:
canal = Canal()
debut = datetime.now()
duree_ms = 150
messages_alice, messages_bob = generer_secret(canal, "alice", "bob", duree_ms)

print(extraire_secret(canal, "alice", messages_alice, debut, duree_ms))
print(extraire_secret(canal, "bob", messages_bob, debut, duree_ms))

10010110010010100110110100
10010110010010100110110100


## Partie 3

### Question 1

* 
* 

### Question 2

Un potentiel adversaire qui serait passif, se contentant de lire les échanges, ne serait pas en mesure de reconstituer car il ignore l’identité des auteurs des messages échangés entre Alice et Bob et donc la valeur, `0` ou `1`, à y associer pour composer la clé.

### Question 3

Cela fait émerger un moyen pour deux entités chiffrer symétriquement les messages qu’ils voudront s’échanger grâce au secret généré, et ce à partir d’un système à priori non sécurisé puisque toutes les communications y sont lisibles en clair.